# Chris Coxen RTI Exercise

In [1]:
import os
import json
import pandas as pd
import numpy as np  
import matplotlib as plt
import string
import nltk
import random
import pyLDAvis
import pyLDAvis.sklearn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk import FreqDist
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

## Task 1: Read in the data and create a new data frame from the json files

In [2]:
# Set the working directory to where all of the json data are stored
os.chdir('G:/My Drive/RTI/data/json')

# Create empty lists for each element of the json that we will fill with the loop below
event_ids = []
narratives = []
probable_causes = []

# Loop through each json file in the directory that contains all of the jsons
for file in os.listdir('G:/My Drive/RTI/data/json'):
    # Open each json file
    opened_json = open(file)           
    # Load it 
    loaded_json = json.load(opened_json)
    # Loop through each element of the loaded json and append that information to our three storage lists event_ids, narratives, probable_causes
    for i in range(len(loaded_json['data'])):
        event_id = loaded_json['data'][i]['EventId']
        narrative = loaded_json['data'][i]['narrative']
        probable_cause = loaded_json['data'][i]['probable_cause']
        event_ids.append(event_id)
        narratives.append(narrative)
        probable_causes.append(probable_cause)

# Zip all of the lists together into one list so we can pass it to a new dataframe
zipped_lists = list(zip(event_ids, narratives, probable_causes))

# Create a dataframe of all of our narrative information and name the columns appropriately 
narratives = pd.DataFrame(zipped_lists, columns = ['EventId', 'Narrative', 'ProbableCause'])

## Join the narratives with the aviation metadata

In [3]:
# Read in the aviation metadata csv
# Set the working directory to where all of the data are stored 
os.chdir('G:/My Drive/RTI/data/csv')

# Read in the aviation CSV
aviation = pd.read_csv('AviationData.csv')

# Join the aviation dataframe with all_narratives that we created above. Now we have one dataframe with all of our data. 
aviation_narratives = pd.merge(aviation, narratives, on = 'EventId', how = 'inner')

# Check on how many narratives we are missing
(aviation_narratives['Narrative'].values == '').sum() # 231 total

# Not many. Let's delete the rows that are missing a narrative. We still have a large sample to work from
aviation_narratives = aviation_narratives[aviation_narratives.Narrative != '']

## Task 2: Exploratory Data Analysis / Descriptive Statistics

In [4]:
# How are the data structured?
aviation_narratives.info()

# Convert the date column to datetime type
aviation_narratives['EventDate'] = pd.to_datetime(aviation_narratives['EventDate'])

# Pull basic descriptive statistics for our integer data (ignore the Lat / Long)
descriptive_stats = aviation_narratives.describe()
print(descriptive_stats)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77026 entries, 1 to 77256
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EventId               77026 non-null  object 
 1   InvestigationType     77026 non-null  object 
 2   AccidentNumber        77026 non-null  object 
 3   EventDate             77023 non-null  object 
 4   Location              76945 non-null  object 
 5   Country               76517 non-null  object 
 6   Latitude              23722 non-null  float64
 7   Longitude             23713 non-null  float64
 8   AirportCode           43300 non-null  object 
 9   AirportName           46027 non-null  object 
 10  InjurySeverity        77021 non-null  object 
 11  AircraftDamage        74747 non-null  object 
 12  AircraftCategory      16496 non-null  object 
 13  RegistrationNumber    74271 non-null  object 
 14  Make                  76938 non-null  object 
 15  Model              

In [5]:
# Pull basic stats for the categorical data of interest

# Aircraft Damage
aviation_narratives['AircraftDamage'].describe()
aviation_narratives['AircraftDamage'].value_counts()

Substantial    55323
Destroyed      16977
Minor           2447
Name: AircraftDamage, dtype: int64

In [6]:
# Aircraft Category
aviation_narratives['AircraftCategory'].describe()
aviation_narratives['AircraftCategory'].value_counts()

Airplane             14190
Helicopter            1771
Glider                 261
Balloon                120
Gyrocraft               62
Unknown                 40
Weight-Shift            20
Ultralight              18
Powered Parachute       10
Powered-Lift             2
Rocket                   1
Blimp                    1
Name: AircraftCategory, dtype: int64

In [7]:
# Aircraft Make - there are multple cases used for these in the original csv (ie CESSNA and Cessna) so we'll make this column all lowercase first
aviation_narratives['Make'] = aviation['Make'].str.lower()
aviation_narratives['Make'].describe()
aviation_narratives['Make'].value_counts()


cessna            24275
piper             13241
beech              4770
bell               2395
boeing             2055
                  ...  
dutton                1
brook                 1
calstar 1             1
vance renegade        1
sons mustang          1
Name: Make, Length: 6431, dtype: int64

In [8]:
# Aircraft Model
aviation_narratives['Model'].describe()
aviation_narratives['Model'].value_counts()

152                   2247
172                   1159
172N                  1117
PA-28-140              900
172M                   769
                      ... 
SR-10E                   1
J4/E                     1
SR-8E                    1
Kitfox 6                 1
WCS-222 (BELL 47G)       1
Name: Model, Length: 10996, dtype: int64

In [9]:
# Amateur Built
aviation_narratives['AmateurBuilt'].describe()
aviation_narratives['AmateurBuilt'].value_counts()

No     69080
Yes     7393
Name: AmateurBuilt, dtype: int64

In [10]:
# Number of Engines
aviation_narratives['NumberOfEngines'].describe()
aviation_narratives['NumberOfEngines'].value_counts()

1.0     61361
2.0      9843
0.0      1124
3.0       475
4.0       406
24.0        1
18.0        1
Name: NumberOfEngines, dtype: int64

In [11]:
# Engine Type
aviation_narratives['EngineType'].describe()
aviation_narratives['EngineType'].value_counts()

Reciprocating       62915
Turbo Shaft          3199
Turbo Prop           2931
Turbo Fan            2142
Unknown              2053
Turbo Jet             666
None                    5
TF, TJ                  3
REC, TJ, TJ             2
Hybrid Rocket           1
Electric                1
REC, TJ, REC, TJ        1
TF, TJ, TJ              1
TJ, REC, REC, TJ        1
Name: EngineType, dtype: int64

In [12]:
# Purpose of Flight
aviation_narratives['PurposeOfFlight'].describe()
aviation_narratives['PurposeOfFlight'].value_counts()

Personal                     43287
Instructional                 9258
Unknown                       6753
Aerial Application            4281
Business                      3825
Positioning                   1485
Other Work Use                1092
Ferry                          767
Public Use                     700
Aerial Observation             643
Executive/Corporate            509
Flight Test                    292
Air Race/Show                  133
Skydiving                      106
Public Aircraft - Federal       83
Banner Tow                      77
External Load                   75
Public Aircraft - State         51
Public Aircraft - Local         49
Glider Tow                      40
Fire Fighting                   19
Air Drop                         8
Name: PurposeOfFlight, dtype: int64

In [13]:
# Air Carrier
aviation_narratives['AirCarrier'].describe()
aviation_narratives['AirCarrier'].value_counts()

UNITED AIRLINES                   49
AMERICAN AIRLINES                 41
CONTINENTAL AIRLINES              25
USAIR                             24
DELTA AIR LINES INC               23
                                  ..
(DBA: TANANA AIR SERVICE)          1
SUNJET AVIATION INC                1
ALLEGHENY AIRLINES INC.            1
FEDEX                              1
ROCKY MOUNTAIN HELICOPTERS, IN     1
Name: AirCarrier, Length: 2807, dtype: int64

In [14]:
# Weather Conditions
aviation_narratives['WeatherCondition'].describe()
aviation_narratives['WeatherCondition'].value_counts()

VMC    68647
IMC     5571
UNK      922
Name: WeatherCondition, dtype: int64

In [15]:
# Broad Phase of Flight
aviation_narratives['BroadPhaseOfFlight'].describe()
aviation_narratives['BroadPhaseOfFlight'].value_counts()

LANDING        18521
TAKEOFF        14724
CRUISE         10577
MANEUVERING     9485
APPROACH        7494
TAXI            2258
CLIMB           2223
DESCENT         2144
GO-AROUND       1568
STANDING        1162
UNKNOWN          614
OTHER            147
Name: BroadPhaseOfFlight, dtype: int64

## Start of text processing

In [25]:
# Examine some of the narratives - can run as many times as you would like to get a sample of narratives
random_row = random.randint(0, len(aviation_narratives))
aviation_narratives.at[random_row, 'Narrative']

'NTSB investigators used data provided by various entities, including, but not limited to, the Federal Aviation Administration and/or the operator and did not travel in support of this investigation to prepare this aircraft accident report.The pilot had been practicing touch-and-go landings.  On his final landing, the airplane swerved to the right and the right wing struck the ground, bending the spar.  The airplane then departed the runway.  According to the FAA airworthiness inspector who examined the airplane, the right brake caliper assembly showed "severely worn shoes with uneven wear."  The inspector surmised that when the pilot applied the brakes,  the uneven brake shoe wear "caused the caliper pins to engage misaligned, contacting the rotor, locking and shearing the caliper assembly."'

The first sentence from most of these is either a boilerplate intro or information that is otherwise provided in the metadata

In [26]:
# Let's delete the first sentence from each narrative 
# A function that deletes the first sentence
def remove_first_sentence(input):
    sentences = nltk.sent_tokenize(input)
    sentences = sentences[1:len(sentences)]
    sentences = ''.join(sentences)
    return sentences

In [45]:
# Create a new column 'ReducedNarrative' that we will create and fill using the narratives without their first sentence
# We will use this column moving forward to analyze and further clean the text
aviation_narratives['ReducedNarrative'] = aviation_narratives['Narrative'].apply(lambda input: remove_first_sentence(input))

# Are there any empty values here because there was only one sentence in the narrative?
print((aviation_narratives['ReducedNarrative'].values == '').sum())

# Remove empty narratives 
aviation_narratives = aviation_narratives[aviation_narratives.ReducedNarrative != '']

0


In [46]:
# Get a frequency distribution of the most common words used in the currently uncleaned narrative 
unprocessed_freq = FreqDist(' '.join(aviation_narratives['ReducedNarrative'].str.lower()).split()).most_common(50)
print(unprocessed_freq)

[('the', 820457), ('and', 228278), ('to', 226538), ('of', 197999), ('was', 171811), ('a', 165246), ('that', 101091), ('pilot', 93030), ('airplane', 90149), ('in', 87369), ('he', 74279), ('on', 62321), ('at', 58557), ('with', 56690), ('were', 47159), ('engine', 45897), ('not', 44653), ('had', 43050), ('from', 40687), ('an', 39510), ('landing', 39387), ('fuel', 38892), ('for', 38402), ('left', 37511), ('flight', 33323), ('right', 31978), ('reported', 31397), ('about', 31023), ('no', 28711), ('aircraft', 26968), ('revealed', 25516), ('runway', 24340), ('during', 24224), ('it', 23919), ('acft', 23425), ('gear', 22881), ('by', 22750), ('or', 22738), ('stated', 22452), ('then', 22385), ('accident', 21597), ('&', 20799), ('feet', 20364), ('as', 19908), ('but', 19603), ('his', 19211), ('after', 19094), ('power', 19022), ('did', 18709), ('which', 17750)]


`There are a lot of uninformative words in here. Let's remove stopwords and lemmatize` 

In [47]:
# Build a function that will lemmatize all of our words
lemmatizer = WordNetLemmatizer()

# First create a function that will get the type of word part of speech. 
# This will help us lemmatize by multiple POS. Lemmatizer will default to only lemmatizing nouns. 
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def clean_text(input):
    # Set our stopwords and also include any punctuation. We will clean based on this variable. 
    stops = set(stopwords.words('english') + list(string.punctuation))
    # Only keep tokens that are not in stops and are alphanumeric text (I noticed that `` occurs as a token after trial and error)
    words = [word for word in pos_tag(word_tokenize(input.lower())) if word[0] not in stops and word[0].isalnum() == True]
    lemmatized_text = []
    for word, pos in words:
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        lemmatized_text.append(lemmatizer.lemmatize(word, pos = wordnet_pos))
    # do not join the text if the token is a digit
    return ' '.join((word for word in lemmatized_text if not word.isdigit()))

In [55]:
# Test it out on some sample text to make sure it behaves like we want it to. Run several times to evaluate performance.
# Generate a random number to sample from
random_row = random.randint(0, len(aviation_narratives))
print('This is the unprocessed text:')
print(aviation_narratives.at[random_row, 'Narrative'])
print('\n')
print('This is the processed text:')
print(clean_text(aviation_narratives.at[random_row, 'ReducedNarrative']))

This is the unprocessed text:
NTSB investigators may not have traveled in support of this investigation and used data provided by various sources to prepare this aircraft accident report.During the takeoff roll, the flightcrew heard a "loud pop," and the airplane began to pull to the left. The pilot flying (PF) aborted the takeoff, and the airplane began to "fishtail" down the runway. The drag chute was activated by the pilot not flying (PNF); however, it appeared to be inoperative and the pilots were unable to stop the airplane on the runway. The airplane continued off the right side, impacted a runway light, and came to rest in the grass. Examination of the airplane revealed that both of the left main landing gear tires had blown, and the left main landing gear was separated from the airplane. Due to severe fragmentation of the tires, the origin of the tire failure could not be identified. Additionally, the strap of the drag chute had separated from the airplane, and was located in t

In [56]:
# Run our function on the CleanNarrative text and overwrite the current column
# This takes a few minutes to run
aviation_narratives['CleanNarrative'] = aviation_narratives['ReducedNarrative'].apply(lambda input: clean_text(input))

In [57]:
aviation_narratives['CleanNarrative'].head()

1    flight instructor private pilot instruction ai...
5    c90 sustain substantial damage falcon sustain ...
6    c90 sustain substantial damage falcon sustain ...
7    pilot two passenger receive minor helicopter s...
8    operate airplane provision code federal regula...
Name: CleanNarrative, dtype: object

## Task 3: Analysis of text frequency over time

In [58]:
# What is the count of entries by year?
aviation_narratives.groupby([aviation_narratives.EventDate.dt.year.rename('year')]).size()

year
1948.0       1
1962.0       1
1974.0       1
1977.0       1
1979.0       1
1981.0       1
1982.0    3489
1983.0    3458
1984.0    3372
1985.0    2985
1986.0    2760
1987.0    2728
1988.0    2658
1989.0    2475
1990.0    2416
1991.0    2398
1992.0    2268
1993.0    2240
1994.0    2240
1995.0    2293
1996.0    2175
1997.0    2136
1998.0    2209
1999.0    2191
2000.0    2210
2001.0    2044
2002.0    2012
2003.0    2078
2004.0    1941
2005.0    2024
2006.0    1836
2007.0    2008
2008.0    1925
2009.0    1799
2010.0    1812
2011.0    1874
2012.0    1840
2013.0    1526
2014.0    1486
2015.0     890
dtype: int64

In [59]:
# What is the count by decade?
aviation_narratives.groupby(pd.cut(aviation_narratives.EventDate, pd.date_range('1940', '2020', freq='10YS'), right=False)).size()

EventDate
[1940-01-01, 1950-01-01)        1
[1950-01-01, 1960-01-01)        0
[1960-01-01, 1970-01-01)        1
[1970-01-01, 1980-01-01)        3
[1980-01-01, 1990-01-01)    23926
[1990-01-01, 2000-01-01)    22566
[2000-01-01, 2010-01-01)    19877
[2010-01-01, 2020-01-01)     9428
dtype: int64

In [60]:
# Separate the data into sub data frames by decade. There are cleaner ways to do this, but this will suffice for the notebook.
# Create a year column
aviation_narratives['Year'] = pd.DatetimeIndex(aviation_narratives['EventDate']).year

In [61]:
# 1940s
narr_1940s = aviation_narratives.loc[(aviation_narratives['Year'] < 1950)]
FreqDist(' '.join(narr_1940s['CleanNarrative']).split()).most_common(5)

[('selway', 2), ('river', 2), ('wreckage', 2), ('two', 1), ('pilot', 1)]

In [62]:
# 1950s - There are no 
narr_1950s = aviation_narratives.loc[(aviation_narratives['Year'] >= 1950) & (aviation_narratives['Year'] < 1960)]
FreqDist(' '.join(narr_1950s['CleanNarrative']).split()).most_common(5)

[]

In [63]:
# 1960s
narr_1960s = aviation_narratives.loc[(aviation_narratives['Year'] >= 1960) & (aviation_narratives['Year'] < 1970)]
FreqDist(' '.join(narr_1960s['CleanNarrative']).split()).most_common(5)

[('airplane', 4),
 ('park', 2),
 ('subsequently', 1),
 ('collide', 1),
 ('rise', 1)]

In [64]:
# 1970s
narr_1970s = aviation_narratives.loc[(aviation_narratives['Year'] >= 1970) & (aviation_narratives['Year'] < 1980)]
FreqDist(' '.join(narr_1970s['CleanNarrative']).split()).most_common(5)

[('pilot', 9), ('aircraft', 5), ('flight', 4), ('foot', 4), ('wreckage', 3)]

In [65]:
# 1980s
narr_1980s = aviation_narratives.loc[(aviation_narratives['Year'] >= 1980) & (aviation_narratives['Year'] < 1990)]
FreqDist(' '.join(narr_1980s['CleanNarrative']).split()).most_common(5)

[('acft', 22967),
 ('plt', 14850),
 ('pilot', 9789),
 ('aircraft', 9074),
 ('fuel', 8545)]

It looks like they start to use acronyms in the narratives - using acft, plt

In [68]:
# 1990s
narr_1990s = aviation_narratives.loc[(aviation_narratives['Year'] >= 1990) & (aviation_narratives['Year'] < 2000)]
FreqDist(' '.join(narr_1990s['CleanNarrative']).split()).most_common(5)

[('pilot', 32334),
 ('airplane', 30251),
 ('engine', 15438),
 ('fuel', 12181),
 ('aircraft', 11937)]

In [69]:
# 2000s
narr_2000s = aviation_narratives.loc[(aviation_narratives['Year'] >= 2000) & (aviation_narratives['Year'] < 2010)]
FreqDist(' '.join(narr_2000s['CleanNarrative']).split()).most_common(5)

[('airplane', 45769),
 ('pilot', 41240),
 ('engine', 19802),
 ('flight', 16279),
 ('report', 14813)]

In [70]:
# 2010s
narr_2010s = aviation_narratives.loc[(aviation_narratives['Year'] >= 2010) & (aviation_narratives['Year'] < 2020)]
FreqDist(' '.join(narr_2000s['CleanNarrative']).split()).most_common(5)

[('airplane', 45769),
 ('pilot', 41240),
 ('engine', 19802),
 ('flight', 16279),
 ('report', 14813)]

`Many of these words are still very generic. We can narrow down the words by frequency using the CountVectorizer` 

## Task 4: Topic clustering with LDA

In [71]:
# Create a count vectorizer so we can perform LDA. LDA needs raw frequency counts and then will weight terms for us. 
# We will set the max_df to something small to exclude words that are frequently used in the text
vectorizer = CountVectorizer(analyzer='word',
                             max_df=0.1, # ignore terms that have a frequency higher than 0.1 (trying to get more specific topics)
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=1000) # max number of unique words

data_vectorized = vectorizer.fit_transform(list(aviation_narratives['CleanNarrative']))

In [115]:
# Take a look at the word / frequency breakdown produced
vectorizer.vocabulary_

{'instructor': 447,
 'private': 669,
 'instruction': 446,
 'sustain': 880,
 'substantial': 867,
 'register': 720,
 'aviation': 81,
 'operate': 589,
 'west': 980,
 'code': 167,
 'federal': 324,
 'regulation': 721,
 'part': 613,
 'meteorological': 527,
 'prevail': 665,
 'instrument': 448,
 'rule': 762,
 'ifr': 417,
 'local': 494,
 'originate': 597,
 'perform': 626,
 'last': 469,
 'depart': 223,
 'experience': 309,
 'loss': 504,
 'conduct': 182,
 'forced': 350,
 'field': 328,
 'beyond': 103,
 'departure': 224,
 'rest': 735,
 'retain': 739,
 'minor': 532,
 'airline': 33,
 'transport': 929,
 'two': 938,
 'passenger': 618,
 'provision': 680,
 'subsequent': 865,
 'preflight': 659,
 'checklist': 156,
 'complete': 175,
 'park': 610,
 'brake': 118,
 'set': 790,
 'handle': 386,
 'pull': 682,
 'wheel': 982,
 'soon': 831,
 'remove': 725,
 'start': 849,
 'across': 9,
 'immediately': 420,
 'apply': 57,
 'however': 411,
 'stop': 857,
 'emergency': 280,
 'system': 883,
 'slow': 824,
 'add': 14,
 'ramp'

In [ ]:
# Find the best number of topics and learning decay to perform LDA
# I commented this out because it took several hours to run

# Define Search Parameters
#In the interest of avoiding long compute time, let's set 12 as the max amount of clusters since more clusters will mean more homogeneity.
# Set a min of 2 clusters since 1 cluster is meaningless.
#search_params = {'n_components': range(2,12)}

# Initialize the model
#lda = LatentDirichletAllocation()

# Initialize a grid search class
#model = GridSearchCV(lda, param_grid=search_params)

# Perform a grid search on our search parameters
#model.fit(data_vectorized)

# Best model
#best_lda_model = model.best_estimator_

# Model parameters
#print("Best Model's Params: ", model.best_params_) # 5 components (aka topics) was the best number 

In [88]:
# Build the LDA Model
lda_model = LatentDirichletAllocation(n_components=5, # Number of topics
                                      learning_method='online', # This will improve speed vs batch   
                                      random_state=100, # Set a random state for output consistency 
                                      n_jobs = -1, # Use all available CPUs
                                     )

lda_output = lda_model.fit_transform(data_vectorized)

In [86]:
# I'd like to see the top keywords for each topic
# A function that will generate the top n keywords for each topic
def show_topics(vectorizer, model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

# What are the top 15 words for each topic cluster?
topic_keywords = show_topics(vectorizer=vectorizer, model=lda_model, n_words=15)  

In [87]:
# Create a topic - keyword dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
0,witness,rwy,mile,passenger,weather,flt,crash,observe,wreckage,contact,cessna,near,level,receive,altitude
1,tank,run,carburetor,loss,forced,field,water,full,gallon,position,system,climb,altitude,flap,selector
2,eng,inspection,oil,fail,propeller,fire,separate,blade,cylinder,maintenance,assembly,evidence,fracture,hour,result
3,helicopter,student,brake,touch,apply,tail,knot,rotor,instructor,stop,full,hard,touchdown,veer,contact


The first topic is kind of ambiguous. It contains weather, but it also contains 'rwy' or runway. It could be a weather related crash while trying to land, but it could also be talking about crashes in general (not helpful)

The second topic seems to be focused on fuel issues - tank, carburetor, loss, gallon - so this one looks better.

The third topic looks to be focused on mechanical issues - engine, oil, fail, propeller, fire - this topic is strong.

The fourth topic may be helicopter specific - helicopter and rotor appear - but it also looks to include student pilots with words like instructor and student. Perhaps it's student helicopter pilots?

The fifth topic seems to talk about steering and landing the plane - apply, touch, brake, stop, flap, touchdown - this topic appears to be pretty strong as well.

In [77]:
# Assign unique names to the topics based on the words present
topics = ['Topic 1: Weather / misc crash', 'Topic 2: Fuel Issue', 'Topic 3: Engine / Mechanical Failure', 'Topic 4: Helicopter / student', 'Topic 5: Steering or Landing']
df_topic_keywords.index = [topics]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 1: Weather / misc,witness,rwy,mile,weather,flt,observe,crash,altitude,level,wreckage,climb,descend,contact,saw,locate
Topic 2: Fuel Issue,tank,run,carburetor,forced,loss,field,cessna,gallon,hour,selector,switch,pump,water,full,position
Topic 3: Mechanical Failure,eng,inspection,oil,fail,propeller,fire,cylinder,system,separate,maintenance,assembly,fracture,evidence,result,number
Topic 4: Helicopter,helicopter,passenger,rotor,instructor,student,cfi,tail,glider,seat,operate,federal,injury,receive,aviation,meteorological
Topic 5: Steering or Landing,apply,touch,brake,full,knot,stop,flap,back,touchdown,wheel,veer,rudder,rest,hit,hard


This is helpful but let's visualize what's going on with our topics. The ideal cluster outcome will be decently sized clusters that are heterogeneous in space. There's a really cool package called pyLDAvis that will visualize our model and allow us to interact with the clusters to see their terms.

## Task 5: Create a chart that you feel conveys one important relationship in the data

In [147]:
import pyLDAvis.sklearn

# From experience this package uses some deprecated functions that throw warnings so we will ignore those warnings below.
import warnings
warnings.filterwarnings('ignore')

# Setting up pyLDAvis
pyLDAvis.enable_notebook()

# This takes our lda model, the word count vectorized 'bag of words', and the vectorizer model,
# reduces the data down to two dimensions for display, and provides a nifty term frequency chart for each cluster. 
# We will use tsne for dimension reduction. 
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne', sort_topics=False)

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  28.204314
1       51.163364 -159.968018       2        1  16.701029
2      -73.144554   21.523815       3        1  20.212031
3      -66.410133 -110.606438       4        1  10.432823
4       54.417324   12.443444       5        1  24.449803, topic_info=            Term          Freq         Total Category  logprob  loglift
395   helicopter  17134.000000  17134.000000  Default  30.0000  30.0000
889         tank  13303.000000  13303.000000  Default  29.0000  29.0000
618    passenger  10152.000000  10152.000000  Default  28.0000  28.0000
756        rotor   7753.000000   7753.000000  Default  27.0000  27.0000
864      student   9451.000000   9451.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
940       unable   3293.438861   6116.761694   Topic5  -5.0009   0.7894
187      contact   3798.098048   9366.496171   Topic5  -4.8583   0.5059
396         high   3419.452171   7351.650211   Topic5  -4.9634   0.6431
867  substantial   2865.256229   4632.763721   Topic5  -5.1402   0.9281
738       result   2891.922068   7506.453076   Topic5  -5.1309   0.4547

[316 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1         5  0.999619           abort
14        1  0.000262             add
14        2  0.194872             add
14        5  0.804665             add
20        1  0.000637  administration
...     ...       ...             ...
990       3  0.003544         witness
993       1  0.745397        wreckage
993       2  0.113607        wreckage
993       3  0.092415        wreckage
993       4  0.048466        wreckage

[602 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

My opinion on these clusters has changed a bit based on the term frequency results for each cluster shown in the visualization.

Topic 1 contains almost all of the 'witness' terms, all 'rwy' terms and all 'weather' terms. This implies this topic may contain the most fatal crashes since it has to do with witness accounts, not first hand accounts. These crashes are likely on runways and weather related. Terms like crash and wreckage support this idea of fatal or serious accidents being captured by this cluster.

Topic 2 pretty much aligns with my original topic name 'Fuel Issue.' It contains the full corpus of tank and carburetor. Words like forced and loss imply losing fuel or having a fuel system related malfunction and being forced to land. 

Topic 3 is similar to Topic 2 in that it aligns well with my original topic name 'Engine / Mechanical Failure.' It contains the full corpus of eng, oil, and cylinder, and words like fail, fire, and propeller support this topic narrative of the engine having serious issues leading to loss of power.

Topic 4 is our smallest cluster but it contains all of the helicopter and instructor word entries. This cluster is obviously related to helicopter incidents - rotor is also here - but this topic looks to be lumping incidents related to pilots that are learning in with helicopter incidents. The prevalence of the acronym cfi (certified flight instructor) and the high frequency of student supports this idea.

Topic 5 largely supports my original name 'Steering or Landing' with its mix of a lot of flight steering related terms. I will also say that my gut feeling is that this topic would not have many fatal crashes given most of the words have to do with in-flight steering issues (meaning it would be a first hand account of the pilot's experience) and the word touchdown is heavily represented, implying the aircraft made it to the ground at least mostly intact. I would assume this topic would capture a lot of minor incidents. 

These topics do have a crude narrative based on term frequency within the selected topic relative to the overall term frequency in the corpus. More advanced techniques like bert may give us better clusters (but admittedly I have never used bert and it's out of my reach for something like this). I did pursue using tfidf, calculating cosine similarity, and using KMeans to cluster, but I couldn't overcome the hurdle of my computer giving up on the cosine similarity math due to the huge size of our matrix. 

This exercise could be done better but I feel good about my results given the time constraints! 